In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from tqdm.notebook import tqdm
import time
import re
import sys
from selenium.webdriver.common.keys import Keys
keys = Keys()

In [70]:
url = 'https://search.shopping.naver.com/search/category?catId=50000003&frm=NVSHCAT&origQuery&pagingIndex=1&pagingSize=40&productSet=total&query&sort=rel&timestamp=&viewType=list' # 네이버 쇼핑 url 입력
driver = webdriver.Chrome("C:/Users/Feldrance/anaconda3/chromedriver.exe")
driver.get(url)
time.sleep(2)
body = driver.find_element_by_tag_name('body')
driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/div[1]/ul/li[2]/a').click() ## 가격비교 분류
time.sleep(2)
driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/div[1]/div/div[1]/a[5]').click() ## 리뷰 많은 순 정렬
time.sleep(1)

for i in range (0,7): # 7회 스크롤, 스크롤 하지 않을 경우 에러 발생
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [71]:
p_btn = ['div[1]', 'div[2]', 'div[3]', 'div[4]', 'div[5]', 'div[6]', 'div[7]', 'div[8]', 'div[9]', 'div[10]',
        'div[11]', 'div[12]', 'div[13]', 'div[14]', 'div[15]', 'div[16]', 'div[17]', 'div[18]','div[19]', 'div[20]',
        'div[21]', 'div[22]', 'div[23]', 'div[24]', 'div[25]', 'div[26]', 'div[27]', 'div[28]', 'div[29]', 'div[30]',
         'div[31]', 'div[32]', 'div[33]', 'div[34]', 'div[35]', 'div[36]', 'div[37]', 'div[38]', 'div[39]', 'div[40]',
        ] # 카테고리 페이지당 제품 40개
rpage_btn = ['a:nth-child(2)', 'a:nth-child(3)', 'a:nth-child(4)', 'a:nth-child(5)', 'a:nth-child(6)',
            'a:nth-child(7)','a:nth-child(8)', 'a:nth-child(9)', 'a:nth-child(10)', 'a:nth-child(11)'
            ] # 리뷰 페이지 10페이지
for i in p_btn:
    a = "//*[@id=\"__next\"]/div/div[2]/div/div[3]/div[1]/ul/div/" + i + "/li/div[1]/div[2]/div[1]/a"
    driver.find_element_by_xpath(a).click() ## 제품 페이지 열기
    time.sleep(2)

    rating_list = [] # 제품 바뀔때마다 리스트 초기화
    review_list = [] # 제품 바뀔때마다 리스트 초기화
    label_list = [] # 제품 바뀔때마다 리스트 초기화
    driver.switch_to.window(driver.window_handles[1]) # 크롤링 할 새 창으로 창 전환
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    pname = soup.find_all('h2')[2].text # 제품명 크롤링 (엑셀 저장시 사용)
    pname = pname.replace("/", "") # 제품명에 '/' 존재시 에러 발생하므로 제거

    for pagenum in rpage_btn:
        driver.find_element_by_css_selector('#section_review > div.pagination_pagination__2M9a4 > '+pagenum).send_keys(keys.ENTER) # 리뷰 페이지 넘김
        time.sleep(2)
        for i in range(0,20):  # 제품 페이지 당 리뷰 20개
            review = soup.find_all('p', class_='reviewItems_text__XIsTc')
            review = review[i].text
            review = re.sub('[^#0-9a-zA-Zr-ㅣ가-힣 ]',"",review) # 불필요한 문자 제거
            review_list.append(review)
            rating = int(soup.find('div', class_='reviewItems_etc_area__2P8i3').get_text()[2]) # 평점
            rating_list.append(rating)
            if int(rating) >= 3:
                label = 1
            else:
                label = 0
            label_list.append(label)
            
driver.close() # 크롤링 종료 시 창 종료
driver.switch_to.window(driver.window_handles[0]) # 기본 카테고리 창으로 전환
res = pd.DataFrame({'ratings': rating_list, 'reviews': review_list, 'label': label_list})
res.to_excel(pname + ' review_crawling.xlsx') # 엑셀로 출력